Задание из 2-х частей. <br>
Берем отызывы за лето (из архива с материалами или предыдущего занятия) <br>
1. Учим conv сеть для классификации - выбить auc выше 0.95 <br>
2. Предобучаем word2vec и его эмбединга инициализируем сетку, как влияет на качество?

### 1

In [1]:
max_words = 2000
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [2]:
import pandas as pd
data = pd.read_excel("отзывы за лето.xls")

In [3]:
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation
import re

exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in exclude]
    return " ".join(txt)

data['text'] = data['Content'].apply(preprocess_text)
data = data[data['Rating'] != 3]
data['target'] = data['Rating'] > 3

In [4]:
data['target'] = data['target'].astype(int)
data.head()

,Rating,Content,Date,text,target
0,5,It just works!,2017-08-14,it just works,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,в целое удобноной приложениеиз минус хотеть сл...,1
2,5,Отлично все,2017-08-14,отлично весь,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать на 1 работа антивирус далёкий н...,1
4,5,"Очень удобно, работает быстро.",2017-08-14,очень удобно работать быстро,1


In [5]:
train_corpus = " ".join(data["text"])
train_corpus = train_corpus.lower()

In [6]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vikvas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [46]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [47]:
tokens_filtered_top[10:20]

['с',
 'что',
 'отлично',
 'спасибо',
 'хороший',
 'нравиться',
 'отличный',
 'это',
 'хорошо',
 'телефон']

In [48]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [49]:
import numpy as np
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [50]:
train = np.asarray([text_to_sequence(text, max_len) for text in data["text"]], dtype=np.int32)

In [51]:
from sklearn.model_selection import train_test_split

In [52]:
X_train, X_test, y_train, y_test = train_test_split(train, data['target'], test_size=0.2,
                                                    random_state=13, stratify=data['target'])

In [53]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [54]:
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [55]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [56]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [57]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
28/28 [==============================] - 1s 39ms/step - loss: 0.4287 - accuracy: 0.8472 - val_loss: 0.3268 - val_accuracy: 0.8475
Epoch 2/20
28/28 [==============================] - 1s 23ms/step - loss: 0.2505 - accuracy: 0.8889 - val_loss: 0.1941 - val_accuracy: 0.9101
Epoch 3/20
28/28 [==============================] - 1s 25ms/step - loss: 0.1705 - accuracy: 0.9291 - val_loss: 0.1693 - val_accuracy: 0.9190
Epoch 4/20
28/28 [==============================] - 1s 24ms/step - loss: 0.1418 - accuracy: 0.9439 - val_loss: 0.1690 - val_accuracy: 0.9196
Epoch 5/20
28/28 [==============================] - 1s 26ms/step - loss: 0.1247 - accuracy: 0.9524 - val_loss: 0.1814 - val_accuracy: 0.9209


In [58]:
score = model.evaluate(X_train, y_train, batch_size=batch_size, verbose=1)
print('\n')
print('Train score:', score[0])
print('Train accuracy:', score[1])

31/31 [==============================] - 0s 6ms/step - loss: 0.1183 - accuracy: 0.9580


Train score: 0.11826689541339874
Train accuracy: 0.9580326676368713


In [59]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

8/8 [==============================] - 0s 6ms/step - loss: 0.1715 - accuracy: 0.9253


Test score: 0.17151686549186707
Test accuracy: 0.9253164529800415


In [60]:
results = model.predict(X_test, batch_size=batch_size, verbose=1)

8/8 [==============================] - 0s 5ms/step


In [61]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test[:, 1], results[:, 1])

0.9668354148908572

### 2

In [62]:
from gensim.models import Word2Vec

In [64]:
model_w2v = Word2Vec(sentences=[train_corpus], size=128, window=5, min_count=2)
embeddings = [np.zeros((1, 128))]
for word in vocabulary:
    if word in model_w2v:
        embedding = model_w2v[word].reshape((1, 128))
    else:
        embedding = np.zeros((1, 128))
    embeddings.append(embedding)
embeddings = np.concatenate(embeddings).astype(np.float32)

<ipython-input-64-488154548fa6>:4: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if word in model_w2v:
<ipython-input-64-488154548fa6>:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  embedding = model_w2v[word].reshape((1, 128))


In [66]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len, weights=[embeddings]))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [67]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [68]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
28/28 [==============================] - 1s 48ms/step - loss: 0.5327 - accuracy: 0.8230 - val_loss: 0.3410 - val_accuracy: 0.8430
Epoch 2/20
28/28 [==============================] - 1s 26ms/step - loss: 0.2522 - accuracy: 0.8766 - val_loss: 0.1831 - val_accuracy: 0.9114
Epoch 3/20
28/28 [==============================] - 1s 26ms/step - loss: 0.1653 - accuracy: 0.9289 - val_loss: 0.1637 - val_accuracy: 0.9203
Epoch 4/20
28/28 [==============================] - 1s 26ms/step - loss: 0.1361 - accuracy: 0.9471 - val_loss: 0.1658 - val_accuracy: 0.9266


In [69]:
score = model.evaluate(X_train, y_train, batch_size=batch_size, verbose=1)
print('\n')
print('Train score:', score[0])
print('Train accuracy:', score[1])

31/31 [==============================] - 0s 7ms/step - loss: 0.1236 - accuracy: 0.9539


Train score: 0.12357057631015778
Train accuracy: 0.9539182186126709


In [70]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

8/8 [==============================] - 0s 7ms/step - loss: 0.1614 - accuracy: 0.9294


Test score: 0.16136613488197327
Test accuracy: 0.9293670654296875


In [71]:
results = model.predict(X_test, batch_size=batch_size, verbose=1)

8/8 [==============================] - 0s 6ms/step


In [72]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test[:, 1], results[:, 1])

0.9679363549889436

In [74]:
X_test[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0, 13])

Даже не пришлось особо ничего подкручивать, результат уже увеличился относительно базовой модели. Наверное, можно ещё лучше сделать, но, честно говоря, мне уже кажется результат потрясающим. В идеале, наверное, можно утяжелить архитектуру модели, в надежде на то, чтобы она стала распознавать сарказм, но, мне кажется, для этого у нас датасет маловат.